In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 12.1MB/s 


# Bulk Testing from .dcm images/file

In [ ]:
import os
import cv2 as cv
import numpy as np
import pydicom as PDCM

def Dicom_to_Image(Path):
    DCM_Img = PDCM.read_file(Path)

    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = DCM_Img.get(0x00281050) #Get window center from tag (0028, 1050)
    Window_Width = DCM_Img.get(0x00281051) #Get window width from tag (0028, 1051)
   # print(Window_Center)
    #Window_Center = str(Window_Center)
    
    #Window_Center = Window_Center[54:57]
    
    #Window_Width = str(Window_Width)
    #Window_Width = (Window_Width[54:58])
    #Window_Center = int(float(Window_Center))
    #Window_Width = int(float(Window_Width))
    #print(Window_Center)
    #print(Window_Width)
    Window_Center = int(DCM_Img.get(0x00281050).value[0]) #Get window center from tag (0028, 1050)
    Window_Width = int(DCM_Img.get(0x00281051).value[0])
  
    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    return New_Img, Instance_Number



In [ ]:
def Dicom_to_Image_MR(Path):
    DCM_Img = PDCM.read_file(Path)

    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = DCM_Img.get(0x00281050) #Get window center from tag (0028, 1050)
    Window_Width = DCM_Img.get(0x00281051) #Get window width from tag (0028, 1051)
    size = DCM_Img.get(0x00280010)
   # print(Window_Center)
    Window_Center = str(Window_Center)
    
    Window_Center = Window_Center[54:57]
    
    Window_Width = str(Window_Width)
    Window_Width = (Window_Width[54:58])
    Window_Center = int(float(Window_Center))
    Window_Width = int(float(Window_Width))
    # print(Window_Center)
    # print(Window_Width)
    # Window_Center = int(DCM_Img.get(0x00281050).value[0]) #Get window center from tag (0028, 1050)
    # Window_Width = int(DCM_Img.get(0x00281051).value[0])
  
    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    return New_Img, Instance_Number



In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import load
from numpy import expand_dims
from matplotlib import pyplot
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image, ImageDraw, ImageFilter
import cv2

#load images array
def load_image(filename, patch_shape):
	# load image with the preferred size
	pixels = load_img(filename, target_size=(patch_shape,patch_shape))
	# convert to numpy array
	pixels = img_to_array(pixels)
	# scale from [0,255] to [-1,1]
	pixels = (pixels - 127.5) / 127.5
	# reshape to 1 sample
	pixels = expand_dims(pixels, 0)
	return pixels

model = load_model("/content/drive/MyDrive/medical_segmentation/output_combined/model_082880.h5") # inpaint model path

inputtype = 'MR'

#Input_Image = '/content/drive/MyDrive/medical_segmentation/test_sets/MR/11/T1DUAL/DICOM_anon/InPhase/IMG-0052-00038.dcm'

Input_Folder = '/content/drive/MyDrive/medical_segmentation/test_sets/MR/4/T1DUAL/DICOM_anon/InPhase'
outputfolder = '/content/drive/MyDrive/medical_segmentation/test_sets/MR/4/T1DUAL/DICOM_anon/InPhase_Prediction/'
for filename in os.listdir(Input_Folder):
  print(filename)
  Input_Image = Input_Folder+'/'+filename

  if inputtype == 'CT':
    Output_Image, Instance_Number = Dicom_to_Image(Input_Image)
    size = 512
  else:
    Output_Image, Instance_Number = Dicom_to_Image_MR(Input_Image)
    
  cv.imwrite('dmctojpg.jpg', Output_Image)
  size = Output_Image.shape[0]
    
  Input = load_image('dmctojpg.jpg',256) #input image path 
  print(Input.shape)
    
  gen_image = model.predict(Input)
  gen_image = (gen_image + 1) / 2.0
  pyplot.imsave("generated.png", gen_image[0]) #output image path
  print(gen_image.shape)
  img = cv2.imread('generated.png')
  b,g,r = cv2.split(img)
  #cv2.imwrite("/content/drive/MyDrive/medical_segmentation/test_sets/MR/11/T1DUAL/DICOM_anon/InPhase_Prediction/"+str(Instance_Number)+".png",r)
  r = cv2.resize(r, (size, size))
  ret, bw_img = cv2.threshold(r,50,255,cv2.THRESH_BINARY)
  orig_image = cv2.imread('dmctojpg.jpg',0)
  orig_image = cv2.resize(orig_image, (size, size))
  print(orig_image.shape)
  print(bw_img.shape)
  newImage = orig_image * (bw_img!=0)
  cv2.imwrite("bw.png",bw_img)
  cv2.imwrite(outputfolder+str(Instance_Number)+".png",newImage)

# print(img.shape)
# img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# print(img.shape)
# ret, bw_img = cv2.threshold(img,146,255,cv2.THRESH_BINARY)

# kernel = np.ones((2, 2), 'uint8')

# dilate_img = cv2.erode(bw_img, kernel, iterations=1)
# kernel = np.ones((3, 3), 'uint8')

# dilate_img = cv2.dilate(dilate_img, kernel, iterations=1)
# cv2.imwrite("dilate.png",dilate_img)
# #kernel = np.ones((3,3),np.uint8)
# #bw_img = cv2.morphologyEx(bw_img, cv2.MORPH_OPEN, kernel)
# im = cv2.imread('dmctojpg.jpg',cv2.COLOR_BGR2GRAY)
# im = cv2.resize(im, (256, 256))
# newImage = im * (dilate_img!=0)

# cv2.imwrite("Binary Image.png",bw_img)

#cv2.imwrite("final.png",newImage)





IMG-0010-00004.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00002.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00016.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00012.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00008.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00006.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00010.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00014.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)
IMG-0010-00018.dcm
(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)


KeyboardInterrupt: ignored

# Single .dcm testing 

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import load
from numpy import expand_dims
from matplotlib import pyplot
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image, ImageDraw, ImageFilter
import cv2

#load images array
def load_image(filename, patch_shape):
	# load image with the preferred size
	pixels = load_img(filename, target_size=(patch_shape,patch_shape))
	# convert to numpy array
	pixels = img_to_array(pixels)
	# scale from [0,255] to [-1,1]
	pixels = (pixels - 127.5) / 127.5
	# reshape to 1 sample
	pixels = expand_dims(pixels, 0)
	return pixels

model = load_model("/content/drive/MyDrive/medical_segmentation/output_combined/model_082880.h5") # inpaint model path

inputtype = 'MR'

#Input_Image = '/content/drive/MyDrive/medical_segmentation/test_sets/MR/11/T1DUAL/DICOM_anon/InPhase/IMG-0052-00038.dcm'

Input_Image = '/content/drive/MyDrive/medical_segmentation/test_sets/MR/4/T1DUAL/DICOM_anon/InPhase/IMG-0010-00030.dcm'
if inputtype == 'CT':
  Output_Image, Instance_Number = Dicom_to_Image(Input_Image)
else:
  Output_Image, Instance_Number = Dicom_to_Image_MR(Input_Image)


cv.imwrite('dmctojpg'+str(Instance_Number)+'.jpg', Output_Image)
size = Output_Image.shape[0]
Input = load_image('dmctojpg'+str(Instance_Number)+'.jpg',256) #input image path 
print(Input.shape)
    
gen_image = model.predict(Input)
gen_image = (gen_image + 1) / 2.0
pyplot.imsave("generated.png", gen_image[0]) #output image path
print(gen_image.shape)
img = cv2.imread('generated.png')
b,g,r = cv2.split(img)
#cv2.imwrite("/content/drive/MyDrive/medical_segmentation/test_sets/MR/11/T1DUAL/DICOM_anon/InPhase_Prediction/"+str(Instance_Number)+".png",r)
r = cv2.resize(r, (size, size))
ret, bw_img = cv2.threshold(r,50,255,cv2.THRESH_BINARY)
orig_image = cv2.imread('dmctojpg'+str(Instance_Number)+'.jpg',0)
orig_image = cv2.resize(orig_image, (size, size))
print(orig_image.shape)
print(bw_img.shape)
newImage = orig_image * (bw_img!=0)
cv2.imwrite("bw.png",bw_img)
cv2.imwrite(str(Instance_Number)+"-predicted.png",newImage)

# print(img.shape)
# img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# print(img.shape)
# ret, bw_img = cv2.threshold(img,146,255,cv2.THRESH_BINARY)

# kernel = np.ones((2, 2), 'uint8')

# dilate_img = cv2.erode(bw_img, kernel, iterations=1)
# kernel = np.ones((3, 3), 'uint8')

# dilate_img = cv2.dilate(dilate_img, kernel, iterations=1)
# cv2.imwrite("dilate.png",dilate_img)
# #kernel = np.ones((3,3),np.uint8)
# #bw_img = cv2.morphologyEx(bw_img, cv2.MORPH_OPEN, kernel)
# im = cv2.imread('dmctojpg.jpg',cv2.COLOR_BGR2GRAY)
# im = cv2.resize(im, (256, 256))
# newImage = im * (dilate_img!=0)

# cv2.imwrite("Binary Image.png",bw_img)

#cv2.imwrite("final.png",newImage)





(1, 256, 256, 3)
(1, 256, 256, 3)
(256, 256)
(256, 256)


True